Ваша задача - написать функцию, которая рассчитывает для каждой переменной значение индекса CSI (characteristic stability index, иногда его еще называют PSI - population stability index).

На вход функции должны подаваться два датасета. В одном из них - значения независимых переменных какой-то ML-модели на выборке разработки, во втором - значения тех же переменных на тестовой выборке. Скачаем эти датасеты:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
X_train = pd.read_csv('https://docs.google.com/uc?id=1--bDvtpHmLmu7JIx5atDZedzkLDe3rzA&export=download')
X_test = pd.read_csv('https://docs.google.com/uc?id=1-2ySFsi0rft_V1lPBNjUswOinHm6uDbk&export=download')

СSI - это показатель стабильности эмпирического распределения, основная идея которого - разбить значения переменной на категории и сравнить долю наблюдений, попавших в каждую категорию на тестовой выборке с долей, попавших в эту категорию на бенчмарке (на выборке разработки). 
Формула расчета индекса CSI: 

\begin{align}
CSI = \sum_{i=1}^K((val_i - dev_i)×ln(\frac{val_i}{dev_i}))
  \end{align}
где K - количество категорий, на которые разбиваются значения переменных, *valᵢ* - доля наблюдений, попавших в категорию i на тестовой выборке, *devᵢ* - доля наблюдений, попавщих в категорию i на выборке разработки.  

Дополнительные требования: 

*   Функция должна возвращать значение индекса CSI, дополнительно можно реализовать вывод долей наблюдений по категориям в каждой из выборок.
*   Числовые переменные функция должна разбивать на K категорий, подбирая границы категорий так, чтобы на выборке разработки доли наблюдений в них были одинаковы. K необходимо вынести в аргументы функции.
*   Для категориальных переменных в качестве категорий используются уникальные значения.
*   Пропущенные значения нужно выносить в отдельную категорию.



In [60]:
# Функция для поиска наибольшего делителя, не большее, чем max_block
def divisor(n, max_block=24): 
  i=2
  pre_i=1
  while((i<max_block)&(n>=i)): 
    while((n%i!=0)&(n>=i)&(i<max_block)):
      i+=1
    if(n%i==0):
      pre_i=i
      i+=1
  if(n%i==0):
    return i
  else:
    if(pre_i==1):
       return divisor(n+1)
    else:
       return pre_i

In [74]:
# Функция для деления на категории по одинаковым долям
def K_share(val, dev, num_bins=0):
   val=val[~np.isnan(val)]
   dev=dev[~np.isnan(dev)]
   count_dev=len(np.unique(dev))
   dev=np.append(dev,10**100)
   dev.sort()
   if(num_bins==0):
     num_bins=divisor(count_dev)
   k = [np.unique(dev)[i] for i in range(0, count_dev+1, int(count_dev/num_bins))]
   dev=dev[:-1]
   if(val.min()<dev.min()):
     k.append(val.min())
   if(val.max()>dev.max()):
     k.append(val.max())
   else:
     k.sort()
     k[-1]=dev.max()
   k.sort()
   k[0] -= 0.01
   k[-1] += 0.01
   return k

Не определилась как необходимо делить на категории из условия, поэтому реализовала две функции: для деления на равные доли по выборке разработки и для деления на равные расстояния по всем показаниям.

In [75]:
# Функция для деления на категории по одинаковым расстояниям
def K(val, dev, num_bins=0):
   val=val[~np.isnan(val)]
   dev=dev[~np.isnan(dev)]
   count_dev=len(np.unique(dev))
   if(num_bins==0):
     num_bins=divisor(count_dev)
   min = np.array([val.min(), dev.min()]).min()
   max = np.array([val.max(), dev.max()]).max()
   k = [min + (max - min)*(i)/num_bins for i in range(num_bins+1)]
   k[0] = min - 0.01
   k[-1] = max + 0.01
   return k

In [76]:
def CSI(val, dev, k):
  def calc_csi(val_i, dev_i):     #расчет CSI
    if(dev_i==0):
      dev_i=10**(-4)
    if(val_i==0):
      val_i=10**(-4)
    return ((val_i - dev_i) * np.log(val_i / dev_i))
 
  def count_i(X, start, end):  #Кол-во элементов, входящих в интервал
    i=0
    count_i=0
    while (X[i]<end):
      if(X[i]>=start):
        count_i+=1
      if(i<len(X)-1):
        i+=1
      else:
        return count_i
    return count_i

  sum_val_i=0
  sum_dev_i=0
  count_val=len(val)
  count_dev=len(dev)
  csi=np.array([])

  val_nan=len(val[np.isnan(val)])/count_val
  dev_nan=len(dev[np.isnan(dev)])/count_dev
  csi=np.append(csi, calc_csi(val_nan, dev_nan))

  val=val[~np.isnan(val)]
  dev=dev[~np.isnan(dev)]
  val.sort()
  dev.sort()

  for i in range(len(k)-1):
    val_i=count_i(val, k[i], k[i+1])/count_val
    sum_val_i+=val_i
    dev_i=count_i(dev, k[i], k[i+1])/count_dev
    print('Доля на тесте val_'+str(i),'{:<20.2%}'.format(val_i), 'Доля на разработке dev_'+str(i), '{:.2%}'.format(dev_i))
    sum_dev_i+=dev_i
    csi=np.append(csi, calc_csi(val_i, dev_i))
    
  print('{:<46}'.format('\nСумма не пустых на тесте sum_val_i ='), '{:.2%}'.format(sum_val_i))
  print('{:<45}'.format('Сумма не пустых на разработке sum_dev_i ='), '{:.2%}'.format(sum_dev_i))
  print('{:<45}'.format('Сумма пустых на тесте val_nan ='),  '{:.2%}'.format(val_nan))
  print('{:<45}'.format('Сумма пустых на разработке dev_nan ='),  '{:.2%}'.format(dev_nan))
  return csi.sum()

In [77]:
# Кодирование категорий
def code_categ(values):
  dict_code_categ={}
  i=0
  for j in set(values):
    dict_code_categ[j]=i
    i+=1
  return dict_code_categ

dict_code_categ=code_categ(np.append(X_train['1'].values,X_test['1'].values))
X_test['1'] = X_test['1'].replace(dict_code_categ)
X_train['1'] = X_train['1'].replace(dict_code_categ)

dict_code_categ=code_categ(np.append(X_train['3'].values,X_test['3'].values))
X_test['3'] = X_test['3'].replace(dict_code_categ)
X_train['3'] = X_train['3'].replace(dict_code_categ)

dict_code_categ=code_categ(np.append(X_train['7'].values,X_test['7'].values))
X_test['7'] = X_test['7'].replace(dict_code_categ)
X_train['7'] = X_train['7'].replace(dict_code_categ)

In [78]:
#CSI при делении на равные доли
csi_arr=[]
for i in X_train.columns:
  print('\n\n', i,':')
  csi=CSI(np.array(X_train[i]), np.array(X_test[i]), K_share(np.array(X_train[i]), np.array(X_test[i])))
  csi_arr.append(csi)
  print('CSI=', csi)



 0 :
Доля на тесте val_0 10.80%               Доля на разработке dev_0 11.87%
Доля на тесте val_1 13.60%               Доля на разработке dev_1 11.87%
Доля на тесте val_2 10.80%               Доля на разработке dev_2 11.87%
Доля на тесте val_3 12.80%               Доля на разработке dev_3 11.87%
Доля на тесте val_4 10.40%               Доля на разработке dev_4 11.87%
Доля на тесте val_5 10.00%               Доля на разработке dev_5 11.87%
Доля на тесте val_6 13.20%               Доля на разработке dev_6 11.87%
Доля на тесте val_7 13.60%               Доля на разработке dev_7 11.87%

Сумма не пустых на тесте sum_val_i =          95.20%
Сумма не пустых на разработке sum_dev_i =     94.93%
Сумма пустых на тесте val_nan =               4.80%
Сумма пустых на разработке dev_nan =          5.07%
CSI= 0.01413596470964694


 1 :
Доля на тесте val_0 3.60%                Доля на разработке dev_0 4.40%
Доля на тесте val_1 24.00%               Доля на разработке dev_1 24.27%
Доля на тесте val_2 2

In [79]:
#CSI при делении на равные расстояния
csi_arr_K=[]
for i in X_train.columns:
  print('\n\n', i,':')
  csi=CSI(np.array(X_train[i]), np.array(X_test[i]), K(np.array(X_train[i]), np.array(X_test[i])))
  csi_arr_K.append(csi)
  print('CSI=', csi)



 0 :
Доля на тесте val_0 1.60%                Доля на разработке dev_0 1.87%
Доля на тесте val_1 7.20%                Доля на разработке dev_1 7.07%
Доля на тесте val_2 17.60%               Доля на разработке dev_2 18.27%
Доля на тесте val_3 26.00%               Доля на разработке dev_3 28.00%
Доля на тесте val_4 22.40%               Доля на разработке dev_4 21.60%
Доля на тесте val_5 13.60%               Доля на разработке dev_5 12.93%
Доля на тесте val_6 6.40%                Доля на разработке dev_6 4.40%
Доля на тесте val_7 0.40%                Доля на разработке dev_7 0.80%

Сумма не пустых на тесте sum_val_i =          95.20%
Сумма не пустых на разработке sum_dev_i =     94.93%
Сумма пустых на тесте val_nan =               4.80%
Сумма пустых на разработке dev_nan =          5.07%
CSI= 0.013202667807936682


 1 :
Доля на тесте val_0 3.60%                Доля на разработке dev_0 4.40%
Доля на тесте val_1 24.00%               Доля на разработке dev_1 24.27%
Доля на тесте val_2 24.4

In [80]:
print(csi_arr)  #CSI при делении на равные доли
print(csi_arr_K)#CSI при делении на равные расстояния

[0.01413596470964694, 0.0022792989548173336, 0.13819443355450284, 0.06056346126257583, 0.0009276576612148333, 0.15456053319035667, 0.04053862438394766, 11.313249307264917, 0.0625444540452426, 0.053222166337642283]
[0.013202667807936682, 0.0022792989548173336, 0.1380671624749767, 0.0038892737040657067, 0.004125232525584619, 0.15601759995556452, 0.037227660939210316, 11.313249307264917, 0.018923947603398803, 0.04804529789316064]
